In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from itertools import combinations
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from zipfile import ZipFile
import warnings
import os
import ast
from scipy.spatial import distance

In [2]:
# used csv files, linking it with their source zip files
csv_to_zip_source = {
    "Netflix_Dataset_Movie.csv": "zip_sources/Netflix_Dataset_Movie.csv.zip",
    "Netflix_Dataset_Rating.csv": "zip_sources/Netflix_Dataset_Rating.csv.zip",
    "tmdb_5000_credits.csv": "zip_sources/tmdb.zip",
    "tmdb_5000_movies.csv": "zip_sources/tmdb.zip"
}

csv_file_names = list(csv_to_zip_source.keys())

# common data directory path
data_dir = "./data"

# if directory does not exist, create it
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# check if all .csv can be found in data directory, if not we extract it from corresponding zip source
for csv_file_name in csv_file_names:
    if not os.path.exists(f"{data_dir}/{csv_file_name}"):
        # if it is not found
        zip_source_file_name = csv_to_zip_source[csv_file_name]
        print(f"❗'{csv_file_name}' does not exist in the '{data_dir}' directory, extracting it from zip file '{zip_source_file_name}'...")
        with ZipFile(zip_source_file_name, 'r') as zip:
            zip.extract(csv_file_name, path=data_dir)
            print(f"Done extracting {csv_file_name} from {zip_source_file_name}")
    else:
        # csv exists and found, let user know
        print(f"'{data_dir}/{csv_file_name}' exists ✅")
    print("---")

'./data/Netflix_Dataset_Movie.csv' exists ✅
---
'./data/Netflix_Dataset_Rating.csv' exists ✅
---
'./data/tmdb_5000_credits.csv' exists ✅
---
'./data/tmdb_5000_movies.csv' exists ✅
---


In [3]:
movies = pd.read_csv(f"{data_dir}/Netflix_Dataset_Movie.csv")
ratings = pd.read_csv(f"{data_dir}/Netflix_Dataset_Rating.csv")
credits = pd.read_csv(f"{data_dir}/tmdb_5000_credits.csv")
tmdb_movies = pd.read_csv(f"{data_dir}/tmdb_5000_movies.csv")
movies

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004,Fidel Castro: American Experience
17767,17768,2000,Epoch
17768,17769,2003,The Company


In [4]:
ratings

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3
...,...,...,...
17337453,520675,3,4496
17337454,1055714,5,4496
17337455,2643029,4,4496
17337456,1559566,3,4496


In [5]:
credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [6]:
tmdb_movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [7]:
df_movies = movies[movies.Name.isin(credits.title)]
df_movies

,Movie_ID,Year,Name
2,3,1997,Character
23,24,1981,My Bloody Valentine
29,30,2003,Something's Gotta Give
44,45,1999,The Love Letter
54,55,1995,Jade
...,...,...,...
17742,17743,2003,Better Luck Tomorrow
17757,17758,1979,Prophecy
17761,17762,1997,Gattaca
17763,17764,1998,Shakespeare in Love


In [8]:
df_ratings = ratings[ratings.Movie_ID.isin(df_movies.Movie_ID)]
df_ratings

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3
...,...,...,...
17324875,15737,2,4492
17324876,502274,2,4492
17324877,1326640,3,4492
17324878,633738,3,4492


---

# Selecting similar users (under Book chapter 9.3/Normalizing ratings and Example 9.9)

---

In [9]:
# Using only movies released after 2000
df_movies_2000 = df_movies[df_movies['Year'] >= 2000]

In [10]:
# Get ratings of movies that came out after the year 2000
df_ratings_2000 = ratings[ratings.Movie_ID.isin(df_movies_2000.Movie_ID)]
df_ratings_2000

,User_ID,Rating,Movie_ID
63663,2473170,5,30
63664,900816,3,30
63665,1990901,4,30
63666,1402412,4,30
63667,1601783,3,30
...,...,...,...
17324875,15737,2,4492
17324876,502274,2,4492
17324877,1326640,3,4492
17324878,633738,3,4492


In [11]:
#Assembling utility matrix, where row represent users (by User_ID), columns are movies (by Movie_ID), and the values are the ratings given to that movie by that user
uti_mx = df_ratings_2000.pivot_table(values='Rating', index='User_ID', columns='Movie_ID', aggfunc='first')

In [12]:
#Normalize utility matrix, subtracting mean of ratings from the actual ratings for each respective user
norm_uti_mx = uti_mx.sub(uti_mx.mean(axis=1), axis=0)

In [13]:
#Calculating Cosine distance of a highlighted user and every other user if they have at least 'n_common_movies' that they both rated

#Choosing highlighted user
chosen_user_id = 6

#Number of common movies needed to be considering other user
n_common_movies = 20

#init matrix
cosine_dist_mx = pd.DataFrame(index=[chosen_user_id], columns=norm_uti_mx.index)
cosine_dist_mx = cosine_dist_mx.drop(6, axis=1)

chosen_set = set(norm_uti_mx.loc[chosen_user_id,:].dropna().index)
for user_id in cosine_dist_mx.columns:
    # list of common movie IDs
    common_list = list(chosen_set & set(norm_uti_mx.loc[user_id,:].dropna().index))
    if len(common_list) > 20:
        a = norm_uti_mx.loc[chosen_user_id, common_list].values
        b = norm_uti_mx.loc[user_id, common_list].values
        cosine_dist_mx[user_id] = distance.cosine(a,b)

C:\Users\Marci\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\Marci\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\Marci\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\Marci\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
C:\Users\Marci\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu *

In [14]:
# sort by distance, lower distance means more similar
cosine_dist_mx_sorted = cosine_dist_mx.sort_values(6, axis=1).dropna(axis=1)

In [15]:
# Matrix with 200 closest users
cosine_dist_mx_sorted.iloc[:, : 200]

User_ID,27061,1527544,468406,1860235,784490,2147661,794999,2322049,723397,1488235,...,1874418,1124875,690000,178393,2617878,682407,47007,1497752,2470184,180802
6,0,0,0,0,0,0,0,0,0,0,...,0.370017,0.370111,0.370213,0.370235,0.370397,0.370482,0.370509,0.370559,0.370782,0.370884


In [16]:
similar_users_by_id = cosine_dist_mx_sorted.iloc[:, : 200].columns
similar_users_by_id

Int64Index([  27061, 1527544,  468406, 1860235,  784490, 2147661,  794999,
            2322049,  723397, 1488235,
            ...
            1874418, 1124875,  690000,  178393, 2617878,  682407,   47007,
            1497752, 2470184,  180802],
           dtype='int64', name='User_ID', length=200)

#### Not used, can be useful later (can be deleted anytime)

In [17]:
# df_ratings_2000.loc[df_ratings_2000["User_ID"] == 7]["Rating"].mean()
# ---
# chosen_user_id = 6
# cosine_dist_mx = pd.DataFrame(index=[chosen_user_id], columns=norm_uti_mx.index)
# a = norm_uti_mx.loc[chosen_user_id, :].values
# for user_id in norm_uti_mx.index:
#     b = norm_uti_mx.loc[user_id, :].values
#     cosine_dist_mx[user_id] = distance.cosine(a,b)
# ---
# 1 - (np.dot(a,b)/(np.linalg.norm(a) * np.linalg.norm(b)))
# ---